In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from pmdarima.datasets import load_ausbeer

# dados trimestrais
series = load_ausbeer(as_series=True)[:-1]

series.plot()

In [ ]:
# divisao entre treino e validacao
from sklearn.model_selection import train_test_split

train, test = train_test_split(series, shuffle=False, test_size=12)

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
# auto arima
import pmdarima as pm

?pm.auto_arima

model = pm.auto_arima(train, m=4, trace=True, suppress_warnings=True)

In [ ]:
model.summary()
# https://github.com/statsmodels/statsmodels/blob/db4336b6ef0de96312fbd7156aab4cb72207f768/statsmodels/tsa/statespace/mlemodel.py#L4598

In [ ]:
from statsmodels.stats.stattools import jarque_bera

?jarque_bera

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

?acorr_ljungbox

In [ ]:
from statsmodels.tsa.stattools import breakvar_heteroskedasticity_test

?breakvar_heteroskedasticity_test

In [ ]:
plot=model.plot_diagnostics(figsize=(15, 12))

In [ ]:
forecasts = model.predict(12)
forecasts

In [ ]:
test = test.rename('Actual')
forecasts = pd.Series(forecasts, index=test.index).rename('Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with Auto Arima')
forecasts.plot(legend=True, color='green')

In [ ]:
# computing the error
from src.error import MASE

err = MASE(train, test, forecasts)

err

In [ ]:
AR = pm.ARIMA(order=(1,0,0))
MA = pm.ARIMA(order=(0,0,1))

AR.fit(train)

In [ ]:
AR.summary()

In [ ]:
plot=AR.plot_diagnostics(figsize=(15, 12))

In [ ]:
MA.fit(train)

MA.summary()

In [ ]:
plot=MA.plot_diagnostics(figsize=(15, 12))

In [ ]:
forecasts = AR.predict(12)
forecasts = pd.Series(forecasts, index=test.index).rename('AR Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with AR')
forecasts.plot(legend=True, color='green')

In [ ]:
err = MASE(train, test, forecasts)

err

## Exponential Smoothing

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
?ETSModel


ses_model_manual = SimpleExpSmoothing(train, initialization_method="heuristic").fit(smoothing_level=0.5, optimized=False)

ses_model_auto = SimpleExpSmoothing(train, initialization_method="estimated").fit()

In [ ]:
ses_model_manual.summary()

In [ ]:
ses_model_auto.summary()

In [ ]:
ses_forecasts = ses_model_auto.forecast(12)

print(MASE(train,test,ses_forecasts))

In [ ]:
ses_forecasts = pd.Series(ses_forecasts, index=test.index).rename('SES Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with SES')
ses_forecasts.plot(legend=True, color='green')

## Método de Holt

In [ ]:
?Holt

holt_model1 = Holt(train, initialization_method="estimated").fit(
    smoothing_level=0.8, smoothing_trend=0.2, optimized=False
)
forecasts_holt_m1 = holt_model1.forecast(12).rename("Holt's linear trend")


holt_model2 = Holt(train, exponential=True, initialization_method="estimated").fit(
    smoothing_level=0.8, smoothing_trend=0.2, optimized=False
)
forecasts_holt_m2 = holt_model2.forecast(12).rename("Exponential trend")


holt_model3 = Holt(train, damped_trend=True, initialization_method="estimated").fit(
    smoothing_level=0.8, smoothing_trend=0.2
)
forecasts_holt_m3 = holt_model3.forecast(12).rename("Additive damped trend")


plt.figure(figsize=(12, 8))
plt.plot(test, marker="o", color="black")
(line1,) = plt.plot(forecasts_holt_m1, marker="o", color="blue")
(line2,) = plt.plot(forecasts_holt_m2, marker="o", color="red")
(line3,) = plt.plot(forecasts_holt_m3, marker="o", color="green")
plt.legend([line1, line2, line3], [forecasts_holt_m1.name, forecasts_holt_m2.name, forecasts_holt_m3.name])

In [ ]:
?ExponentialSmoothing
# Holt’s Winters Seasonal Exponential Smoothing
es = ExponentialSmoothing(
    train,
    seasonal_periods=4,
    trend="add",
    seasonal="add",
    use_boxcox=False,
    initialization_method="estimated",
).fit()

forecasts_es = es.forecast(12).rename("Holt’s Winters Seasonal Exponential Smoothing")

In [ ]:
print(MASE(train,test,forecasts_es))

forecasts_es = forecasts_es.rename('ES Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with ES')
forecasts_es.plot(legend=True, color='green')

In [ ]:
ets = ETSModel(
    train,
    seasonal_periods=4,
    trend="mul",
    seasonal="mul",
    initialization_method="estimated",
).fit()

forecasts_ets = ets.forecast(12).rename("ETS")

In [ ]:
ets.summary()

In [ ]:
print(MASE(train,test,forecasts_ets))

forecasts_ets = forecasts_ets.rename('ETS Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with ETS')
forecasts_ets.plot(legend=True, color='green')

## Naive

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

?NaiveForecaster

naive = NaiveForecaster(strategy='mean')
naive.fit(train)

forecasts_naive = naive.predict(fh=[i for i in range(1,12)])

forecasts_naive

In [ ]:
print(MASE(train,test,forecasts_naive))

forecasts_naive = forecasts_naive.rename('Naive Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with Naive')
forecasts_naive.plot(legend=True, color='green')

In [ ]:
naive = NaiveForecaster(strategy='last', sp=4)
naive.fit(train)

forecasts_naive = naive.predict(fh=[i for i in range(1,12)])

print(MASE(train,test,forecasts_naive))

forecasts_naive = forecasts_naive.rename('Seasonal Naive Forecasts')
ax = train.plot()
ax = test.plot(legend=True,color='red', figsize=(12,6),title='Forecasting with Seasonal Naive')
forecasts_naive.plot(legend=True, color='green')